In [1]:
import sympy as sp
import numpy as np
import math

# var global
variables_name=[]
dictionaire_auxiliary_variable={}
res=1

In [2]:
#créer variable à partir d'un variable name en string, check le nom n'est pas déjà utilisé

#variales_name: une liste global contient des string qui sont le nom des variables, sert à conserver tout les nom des variables déjà créés
def create_variables(var_name):
    if isinstance(var_name, str) :#and var_name not in variables_name:
        try:
            variables_name.append(var_name)
            symbol = sp.symbols(var_name)
            globals()[var_name] = symbol
        except ValueError:
            print(f"{var_name} ValueError")
    return symbol

Définir length de p et q

In [3]:

# N = 15(4) = 3*5  

def define_length(N):
    # taille de N en binaire
    length_N = int(math.log2(N)) + 1
    # length de p = l_1, length de q = l_2

    if N == 15 :
        l_1 = 2
        l_2 = 3
    else:
        k = (length_N) // 2
        l_1 = k
        if length_N % 2 ==0:
            l_2 = k
        else:
            l_2 = k + 1
    return l_1,l_2

In [4]:
#p(pl_1-1,..p1,1)
#q(ql_2-1,..,q_1,1)

def set_variable(N):
    (l_1,l_2) = define_length(N)
    #p
    expr_p = 1
    for i in range(1,l_1):
        var_name = ('p_'+ str(i))
        var = create_variables(var_name)
        expr_p = expr_p + 2**i * var
    #q
    expr_q = 1
    for i in range(1,l_2):
        var_name = ('q_'+ str(i))
        var = create_variables(var_name)
        expr_q = expr_q + 2**i * var
    return (expr_p,expr_q)

In [5]:
#variables_name=[]

# f() = (N-pq)²
def define_function(N):
    expr_pq = 1
    pq_variable = set_variable(N)
    for item in pq_variable:
        expr_pq = expr_pq * item
    return (N - expr_pq)**2

expr = define_function(35)
expr


(-(2*p_1 + 4*p_2 + 1)*(2*q_1 + 4*q_2 + 1) + 35)**2

In [6]:
# Step 1 réduction l'expression de la fonction en réduissant le power des variable 
# Par exemple p_1**2 à p_1 tout simple car 1 * 1 = 1 & 0 * 0 = 0

def reduct_variable_power_term(term):
    new_term = 1
    for variable,power in term.as_powers_dict().items():
        new_term *= variable
    return new_term

def reduct_variable_power_function(expr):
    expanded_expr = sp.expand(expr)
    expanded_reducted = 0
    for term in expanded_expr.args:
        new_term = reduct_variable_power_term(term)
        expanded_reducted += new_term
    return expanded_reducted

#expr_reduct_power = reduct_variable_power_function(expr)
#expr_reduct_power


In [7]:
# Definir une function de key pour sort
def sort_key(variable):
    name = str(variable)
    # Split le variable name aux deux parties: partie letter & parite number 
    letter = name[0]
    number = int(name.split('_')[1])
    return (letter, number)


In [8]:
# Step 2 réduire les termes qui ont une dimension plus que 2
# 
#dictionaire_auxiliary_variable={}
#res=1
def rest_term(coef , rest_variable_liste):
    new_term = coef
    while(len(rest_variable_liste)!=0):
        new_term = new_term * rest_variable_liste[0]
        rest_variable_liste.pop()
    return new_term


# si #variables > 2 : diemension -1
def reduct_demension_term(coef,variables_liste):    
    global res

    # sélection les 3 premières variables
    [x_1, x_2, x_3] = variables_liste[:3]
    
    # garder le reste des variables
    rest_variables_liste= variables_liste[3:]
    new_expr = rest_term(coef , rest_variables_liste)

    # commence la réduction dimension
    tup=(x_1,x_2)
    key = next((k for k, v in dictionaire_auxiliary_variable.items() if v == tup), None)
    if key==None:
        var_name = "x_" + str(res)
        x_4 = create_variables(var_name)
        dictionaire_auxiliary_variable[x_4]=(x_1,x_2)
        res += 1 
    else:
        x_4 = key 

    expr_reduct = x_4 * x_3 + 2 * (x_1 * x_2 - 2 * x_1 * x_4 - 2 * x_2 * x_4 + 3 * x_4)

    expr_reduct = new_expr * expr_reduct
    return sp.expand(expr_reduct)


In [9]:

##
def reduct_demension_function(expr):
    # expr = reduct_variable_power_function(expr)

    # stocker l'espression apres la réduction
    new_expr = 0

    # initialiser le variable
    need_reduction = False


    for term in expr.args:
        
        # get coefficant le constant et tout les variables en les mettant dans une liste
        variables_liste = sorted(term.free_symbols, key=sort_key)
        coef = term.as_coeff_mul()[0]

        # pour les termes qui ont plus de 2 variables, il faut appliquer la formule de réduction
        if len(variables_liste) > 2 :
            new_term = reduct_demension_term(coef , variables_liste) 
            new_expr +=  new_term
            if len(variables_liste) > 3 :  
                need_reduction = True
        else:
            # sinon, on ne fait rien et les met directement dans la nouvelle expression
            new_expr += term

    if need_reduction:
        return reduct_demension_function(new_expr)
    else:
        return new_expr

#reduct_demension_expr = reduct_demension_function(expr_reduct_power)
#reduct_demension_expr

In [10]:
# Step3 Transferer d'Ising à Qubo:
# a * x_i = a * (s_i + 1) / 2
# (a * x_i * x_j) = a * (s_i + 1) * (s_j + 1) / 4
# f(s_n)

def build_h(expr):
    #len de h est le nombre de variables 
    h=()
    for term in expr.args:
        if len(term.free_symbols) == 1:
            h = h + (term.as_coeff_mul()[0] / -2,)
    return h

def transfers(reduct_demension_expr):    
    terms = reduct_demension_expr.args 
    # Update le coef des termes avec seulement des termes avec dimension 1.
    new_expr = 0
    # Matrice contient les coef des termes avec dimension 2. 
    matrix_J = np.zeros((len(variables_name), len(variables_name)), dtype=int)
    for term in terms:
        if len(term.free_symbols) == 2:
            # 2-variables : créer la matrice J
            list_variable_term = sorted(term.free_symbols, key=sort_key)
            coef = term.as_coeff_mul()[0]/4
            i = variables_name.index(list_variable_term[0].name)
            j = variables_name.index(list_variable_term[1].name)
            matrix_J[i][j] = coef/2
            # des 1-variables terms se produissent
            new_expr += coef * list_variable_term[0] + coef * list_variable_term[1]
        else:
            # 1-variable : reconstruire une expression pour calculer le coef des termes
            new_expr += term/2

    tuple_h = build_h(new_expr)

    return tuple_h,matrix_J

#transfers(reduct_demension_expr)



sum up fonction

In [11]:
def get_h_and_J(N):
    #start
    expr = define_function(N)
    expr_reduct_power = reduct_variable_power_function(expr)
    reduct_demension_expr = reduct_demension_function(expr_reduct_power)
    return transfers(reduct_demension_expr)

In [12]:
variables_name=[]
dictionaire_auxiliary_variable={}
res=1
get_h_and_J(15)

((58, 50, 12, -80),
 array([[  0,  25,  -6, -64],
        [  0,   0,   2, -64],
        [  0,   0,   0,  16],
        [  0,   0,   0,   0]]))

In [ ]:
# f() = (N-pq)²
set_variable(N)

In [ ]:
def N_bin(N):
    for i in range(2,len(bin(N))):
        print(bin(N)[i])

N_bin(15)        

1
1
1
1


In [ ]:
# définir l'expression
create_variables('q_1','q_2','p_1','p_2','c_1','c_2','c_3','c_4')
expr = (2*p_2+ 2*p_1*q_1+ 2*q_2- 8*c_2- 4*c_1+ p_1+ q_1- 3)**2 + (2*q_1 + 2*p_2*q_2+ 2*p_1+ 2*c_2- 8*c_4- 4*c_3+ p_2*q_1+ p_1*q_2+ c_1+ 1)**2 + (q_2+ p_2+ c_3+ 2*c_4- 2)**2

In [ ]:
expr

(c_3 + 2*c_4 + p_2 + q_2 - 2)**2 + (-4*c_1 - 8*c_2 + 2*p_1*q_1 + p_1 + 2*p_2 + q_1 + 2*q_2 - 3)**2 + (c_1 + 2*c_2 - 4*c_3 - 8*c_4 + p_1*q_2 + 2*p_1 + p_2*q_1 + 2*p_2*q_2 + 2*q_1 + 1)**2

In [ ]:
# expand
expanded_expr = sp.expand(expr)

ternary_terms = {}
# parcourir chaque terme
for term in expanded_expr.args:
    # coeffcient du terme
    coeff = term.as_coeff_mul()[0]
    # variable du tarme
    variables = term.as_coeff_mul()[1]
    # nombre des variables du terme
    num_vars = len(set(sp.flatten(variables)))
    if num_vars == 3:
        # mettre le terme dans un dictionnaire s'il contient 3 variables
        ternary_terms[sp.Mul(*variables)] = coeff

for term, coeff in ternary_terms.items():
    print(f"{term}, {coeff}")

c_2*p_1*q_1, -32
c_4*p_2*q_2, -32
c_1*p_1*q_1, -16
c_3*p_2*q_2, -16
c_4*p_1*q_2, -16
c_4*p_2*q_1, -16
c_3*p_1*q_2, -8
c_3*p_2*q_1, -8
c_1*p_1*q_2, 2
c_1*p_2*q_1, 2
c_1*p_2*q_2, 4
c_2*p_1*q_2, 4
c_2*p_2*q_1, 4
p_1*p_2*q_2**2, 4
p_2**2*q_1*q_2, 4
c_2*p_2*q_2, 8
p_1*p_2*q_2, 8
p_2*q_1*q_2, 8
p_1*p_2*q_1, 12
p_1*q_1*q_2, 12


In [ ]:
# créer new varibale for a term & return a list of the variables in a term
res=1
def add_new_variable_for_term(term):
    symbol_list = list(term[0].free_symbols)
    var_name = 'x_' + str(res)
    if var_name not in variables_name:
        variables_name.append(var_name)
        symbol = sp.symbols(var_name)
        globals()[var_name] = symbol
        symbol.append(symbol)
        res=res+1
    return symbol_list    

In [ ]:
def substitution_positive_terme(term):
    vars = add_new_variable_for_term()

def substitution_negative_terme(term):
    # on suppose que new_var = item 
    vars = add_new_variable_for_term()
    
    

In [ ]:
def substitution_terme(ternary_terms):
    for term, coeff in ternary_terms.items():
        if coeff > 0:
            new_term = substitution_positive_terme(term)
        else:
            new_term = substitution_negative_terme(term)
    return new_term

### On prend N=143 comme exemple. On utilise la formule qui represnet la relation entre les variable en utilisant la méthode de Table dde Multp.

In [ ]:
#Formule de réduction de dimensionnalité
x_1 = sp.symbols('x_1')
x_2 = sp.symbols('x_2')
x_3 = sp.symbols('x_3')
x_4 = sp.symbols('x_4')

substitution_formula = x_4 * x_3 + x_1 * x_2 - x_1 * x_4 - x_2 * x_4 + x_4
